In [2]:
text = 'regarding the number of parameters for each model the results are not that tight. For fully connected layers, each neuron was one parameter for each input, plus the bias so for the first layer we get (784+1)*500=392.500. The, each layer adds (500+1)*500=250.500 parameters and the softmax layer adds (500+1)*10=5.010 parameters. This works out to be 397.510+250.500*n where n is the number of layers. We can plot these along with the respective accuracies '

# Split the text at every period and reformat with newlines
formatted_text = "\n".join(sentence.strip() + "." for sentence in text.split(".") if sentence.strip())

# Print the result
print(formatted_text)

regarding the number of parameters for each model the results are not that tight.
For fully connected layers, each neuron was one parameter for each input, plus the bias so for the first layer we get (784+1)*500=392.
500.
The, each layer adds (500+1)*500=250.
500 parameters and the softmax layer adds (500+1)*10=5.
010 parameters.
This works out to be 397.
510+250.
500*n where n is the number of layers.
We can plot these along with the respective accuracies.


References: 

Khee, L. K., 2022 \textit{Limitation of Data Visualisation} [Online]. Available from https://medium.com/@e0673935/limitation-of-data-visualization-1e4db068a22a [Accessed 13 October 2024]

Mackiewicz, A., Ratajczak, W., 1993. Principal Components Analysis. \textit{Computers & Geosciences}, 19(3), 303-342